In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1A.9_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1A.9_2019.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "32.22494345718902,736.2495858961457,569.9486268174476,61.91823397087313"
                              ],
                          columns=[
                              "49.0588691437803,150,191.4257835218094,235.6749596122779,277.0383198707593,327,374,418,465,510.7894022617125"
                              ],
                          split_text=True,
                          pages="1-10"
                         )

In [6]:
tables

<TableList n=10>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [9]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES' and table.df.iloc[0,1] != 'AGRA'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)' and table.df.iloc[-1,1] != 'TOTAL (CITIES)'):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [10]:
# Splits the second column into 1st column index and 2nd column test when faulty/required
def split_by_first_space(df, strIndex, stpIndex):
    df = df.iloc[strIndex:stpIndex+1]
    for index, each in df.iterrows():
        space = each[1].find(' ')
        each[0] = each[1][:space]
        each[1] = each[1][space:]

In [11]:
def mergeRows(df, strIndex, stpIndex):
    DF = df.loc[strIndex:stpIndex]
    df = DF.T
    l = []
    for index, each in df.iterrows():
        output = ''
        for ind, cell in each.iteritems():
            output += (str(cell) + ' ')
#         if(output == ''):
#             continue
        l.append(output.strip())
    l = pd.Series(l,name=str(strIndex))
    DF.loc[strIndex] = l

In [12]:
for each_table in tables:
    clean(each_table)

In [13]:
states = [ each.df.copy() for ind,each in enumerate(tables) if(ind%2==0)]
cities = [ each.df.copy() for ind,each in enumerate(tables) if(ind%2==1)]

In [14]:
# List of Column headers as column are nested, i.e exhibit a multi level/index structure
# The multi level structre will be generated and applied in the next 
column_header = [
    ['Dangerous or Careless Driving/Over-taking/etc.','Over Speeding','Driving under Influence of Drug/Alcohol'],
    ['Physical Fatigue of Drivers','Defect in Mechanical Condition of Motor Vehicle','Animal Crossing'],
    ['Weather Condition (Total)','Weather Condition(Poor Visibility)','Weather Condition (Others Causes)'],
    ['Lack of Road Infrastructure','Vehicles Parking at Road Shoulders','Causes Not Known'],
    ['Other Causes','Total Road Accidents','Unmanned Railway Crossing Accidents']
]
len(column_header)

5

In [15]:
def set_Column(df, title, city):
    column_list = [
        ("Sl. No.",''),
        ("State/UT" if(not city) else "City",''), 
        (title[0],'Cases'),
        (title[0],'Injured'),
        (title[0],'Died'),
        (title[1],'Cases'),
        (title[1],'Injured'),
        (title[1],'Died'),
        (title[2],'Cases'),
        (title[2],'Injured'),
        (title[2],'Died')
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [16]:
for i,(city,state) in enumerate(zip(cities,states)):
    set_Column(cities[i], column_header[i],True)
    cities[i].set_index(["Sl. No.","City"],inplace=True)
    set_Column(states[i], column_header[i],False)
    states[i].set_index(["Sl. No.","State/UT"],inplace=True)

In [17]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [18]:
table11_12 = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "34.14882067851374,712.2006789087965,548.7859773828757,84.04322839923444"
                              ],
                          columns=[
                              "63.487948303715676,217.39812600969307,314.5539256865913,437.68206785137323"
                              ],
                              row_tol=10,
                          split_text=True,
                          pages="11-12"
                         )
clean(table11_12[1])

<Table shape=(55, 5)>

In [19]:
def set_Column_last(df, city):
    column_list = [
        ("Sl. No.",''),
        ("State/UT" if(not city) else "City",''), 
        ("Grand Total",'Cases'),
        ("Grand Total",'Injured'),
        ("Grand Total",'Died')
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [20]:
set_Column_last(table11_12[0].df, False)
table11_12[0].df.set_index(["Sl. No.","State/UT"],inplace=True)
set_Column_last(table11_12[1].df, True)
table11_12[1].df.set_index(["Sl. No.","City"],inplace=True)

In [21]:
cities.append(table11_12[1].df.copy())
states.append(table11_12[0].df.copy())

In [22]:
# Set Uniform Indexes across all respective tables
for city,state in zip(cities,states):
    city.index  = cities[0].index.copy()
    state.index = states[0].index.copy()

In [23]:
City  = pd.concat(cities,axis=1)
State = pd.concat(states,axis=1)

In [24]:
City

Dangerous or Careless Driving/Over-taking/etc.  \
                                                                    Cases   
Sl. No. City                                                                
37      AGRA                                                          208   
38      AHMEDABAD                                                       1   
39      ALLAHABAD                                                     107   
40      AMRITSAR                                                       16   
41      ASANSOL                                                       429   
42      AURANGABAD                                                      8   
43      BENGALURU                                                     438   
44      BHOPAL                                                       1532   
45      CHANDIGARH (CITY)                                              14   
46      CHENNAI                                                       645   
47      COIMBATORE                                                     45   
48      DELHI (CITY)                                                 1978   
49      DHANBAD                                                        65   
50      DURG BHILAINAGAR                                                0   
51      FARIDABAD                                                     242   
52      GHAZIABAD                                                      86   
53      GWALIOR                                                       118   
54      HYDERABAD                                                     951   
55      INDORE                                                       3012   
56      JABALPUR                                                      112   
57      JAIPUR                                                       1460   
58      JAMSHEDPUR                                                    191   
59      JODHPUR                                                       121   
60      KANNUR                                                        340   
61      KANPUR                                                        289   
62      KOCHI                                                         763   
63      KOLKATA                                                         3   
64      KOLLAM                                                          0   
65      KOTA                                                           16   
66      KOZHIKODE                                                     555   
67      LUCKNOW                                                       262   
68      LUDHIANA                                                      152   
69      MADURAI                                                        41   
70      MALAPPURAM                                                    216   
71      MEERUT                                                        184   
72      MUMBAI                                                        318   
73      NAGPUR                                                        723   
74      NASIK                                                          36   
75      PATNA                                                         102   
76      PUNE                                                          141   
77      RAIPUR                                                         24   
78      RAJKOT                                                          0   
79      RANCHI                                                        152   
80      SRINAGAR                                                        0   
81      SURAT                                                          93   
82      THIRUVANANTHAPURAM                                            534   
83      THRISSUR                                                      435   
84      TIRUCHIRAPPALLI                                                39   
85      VADODARA                                                       40   
86      VARANASI       

In [25]:
State

Dangerous or Careless Driving/Over-taking/etc.  \
                                                                    Cases   
Sl. No. State/UT                                                            
        STATES                                                              
1       ANDHRA PRADESH                                               5899   
2       ARUNACHAL PRADESH                                              59   
3       ASSAM                                                        1775   
4       BIHAR                                                        2450   
5       CHHATTISGARH                                                 2604   
6       GOA                                                          1594   
7       GUJARAT                                                      1821   
8       HARYANA                                                      3541   
9       HIMACHAL PRADESH                                             1478   
10      JAMMU & KASHMIR                                               616   
11      JHARKHAND                                                    1616   
12      KARNATAKA                                                    4741   
13      KERALA                                                      15802   
14      MADHYA PRADESH                                              15325   
15      MAHARASHTRA                                                 11234   
16      MANIPUR                                                        95   
17      MEGHALAYA                                                     120   
18      MIZORAM                                                        22   
19      NAGALAND                                                        5   
20      ODISHA                                                        438   
21      PUNJAB                                                       1302   
22      RAJASTHAN                                                    6945   
23      SIKKIM                                                         35   
24      TAMIL NADU                                                   5462   
25      TELANGANA                                                    3468   
26      TRIPURA                                                       128   
27      UTTAR PRADESH                                               18886   
28      UTTARAKHAND                                                   271   
29      WEST BENGAL                                                  2605   
        TOTAL (STATES)                                             110337   
        UNION  TERRITORIES                                                  
30      A & NISLANDS                                                   96   
31      CHANDIGARH                                                     14   
32      D & N HAVELI                                                   51   
33      DAMAN & DIU                                                    13   
34      DELHI (UT)                                                   1978   
35      LAKSHADWEEP                                                     1   
36      PUDUCHERRY                                                     29   
        TOTAL (UTs)                                                  2182   
        TOTAL (ALL INDIA)                                          112519   

                                          Over Speeding                 \
                           Injured   Died         Cases Injured   Died   
Sl. No. State/UT                                                         
        STATES                                                           
1       ANDHRA PRADESH        6807   2277         12866   14570   5123   
2       ARUNACHAL PRADESH       49     23            76      97     41   
3       ASSAM                 1577    823          4440    3855   1780   
4       BIHAR                 1630   1862          4766    3510   3387   
5       CHHATTISGARH          2454   1131     

In [26]:
# Reset Index before saving file, for better formatting in RAW CSV
State.reset_index().to_csv("Cause – wise Distribution of Road Accidents and Unmanned Railway Crossing Accidents during 2019(StateUT-wise).csv",index=False)
City.reset_index().to_csv("Cause – wise Distribution of Road Accidents and Unmanned Railway Crossing Accidents during 2019(City-wise).csv",index=False)